# Ensemble Learning

## Initial Imports

In [367]:
import warnings
warnings.filterwarnings('ignore')

In [368]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [369]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [370]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [371]:
# Create our features
x_cols = [i for i in df.columns if i not in ("home_ownership", "next_pymnt_d", "verification_status", "initial_list_status", "issue_d", "loan_status", "pymnt_plan", "application_type", "hardship_flag", "debt_settlement_flag", "pub_rec_bankruptcies")]
X = df[x_cols]
# X.head()
# Create our target
y = df["pub_rec_bankruptcies"]
# y = df["pct_tl_nvr_dlq"]
# y.head()
# y = df["Outcome"]
# X = df.drop(columns="Outcome")
X = df.drop(columns=(["pct_tl_nvr_dlq", "home_ownership", "next_pymnt_d", "verification_status", "initial_list_status", "issue_d", "loan_status", "pymnt_plan", "application_type", "hardship_flag", "debt_settlement_flag", "pub_rec_bankruptcies"]))
# X.astype('int64').dtypes
X.head()
# X.to_csv('LoanStat.csv')

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,percent_bc_gt_75,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0.0,0.0,0.0,3.0,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0.0,0.0,0.0,6.0,50.0,0.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0.0,0.0,0.0,4.0,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0.0,0.0,0.0,1.0,50.0,0.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0.0,0.0,0.0,1.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0


In [372]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,percent_bc_gt_75,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.0,0.0,0.052138,2.219423,30.626217,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.0,0.390633,1.897432,33.631463,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.0,0.0,0.000000,1.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.0,0.0,0.000000,2.000000,20.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.0,0.0,0.000000,3.000000,50.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,0.0,0.0,18.000000,19.000000,100.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [373]:
# Check the balance of our target values
df['pub_rec_bankruptcies'].value_counts()

0.0    60247
1.0     8483
2.0       77
3.0        8
4.0        2
Name: pub_rec_bankruptcies, dtype: int64

In [374]:
# Split the X and y into X_train, X_test, y_train, y_test
# Splitting into Train and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [375]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [376]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [377]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [378]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
# from imblearn.ensemble import RandomForestRegressor

brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [379]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

# y_pred_rf = brf.predict(X_test)
# print(classification_report_imbalanced(y_test, y_pred_rf))

0.4

In [380]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[10776,   693,   307,  2054,  1229],
       [ 1056,   414,    99,   376,   172],
       [    7,     4,     5,     7,     2],
       [    2,     0,     0,     0,     1],
       [    1,     0,     0,     0,     0]], dtype=int64)

In [381]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
# print(classification_report_imbalanced(y_test, y_pred))

y_pred = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.91      0.72      0.50      0.80      0.60      0.37     15059
        1.0       0.37      0.20      0.95      0.26      0.43      0.17      2117
        2.0       0.01      0.20      0.98      0.02      0.44      0.18        25
        3.0       0.00      0.00      0.86      0.00      0.00      0.00         3
        4.0       0.00      0.00      0.92      0.00      0.00      0.00         1

avg / total       0.84      0.65      0.56      0.73      0.58      0.34     17205



In [382]:
# List the features sorted in descending order by feature importance
importances_sorted = sorted(zip(brf.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.02482111741371, 'total_bal_ex_mort'),
 (0.02403329255181107, 'mo_sin_old_rev_tl_op'),
 (0.023050828606384164, 'loan_amnt'),
 (0.022971384082495192, 'total_bc_limit'),
 (0.02277277277277277, 'total_rev_hi_lim'),
 (0.021652604985938323, 'total_il_high_credit_limit'),
 (0.021536086350901172, 'num_bc_tl'),
 (0.02026894619487212, 'mths_since_recent_inq'),
 (0.020239816536112832, 'num_il_tl'),
 (0.02005577005577006, 'mo_sin_old_il_acct')]

### Easy Ensemble Classifier

In [383]:
# Train the Classifier
# from imblearn.ensemble import BalancedRandomForestClassifier
# from imblearn.ensemble import RandomForestRegressor

# brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
rf = RandomForestClassifier(n_estimators=1000, random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=1)

In [384]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.2222290237206807

In [385]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = rf.predict(X_test)
confusion_matrix(y_test, y_pred)


array([[15059,     0,     0,     0,     0],
       [    0,  2117,     0,     0,     0],
       [    0,    25,     0,     0,     0],
       [    0,     3,     0,     0,     0],
       [    0,     1,     0,     0,     0]], dtype=int64)

In [386]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
# print(classification_report_imbalanced(y_test, y_pred))

y_pred = rf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       1.00      1.00      1.00      1.00      1.00      1.00     15059
        1.0       0.99      1.00      1.00      0.99      1.00      1.00      2117
        2.0       0.00      0.00      1.00      0.00      0.00      0.00        25
        3.0       0.00      0.00      1.00      0.00      0.00      0.00         3
        4.0       0.00      0.00      1.00      0.00      0.00      0.00         1

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [ ]:
# Balance Random Forest has best balanced accuracy score. Random Forest has best recall and geometric mean score. Top three features are total_bal_ex_mort, mo_sin_old_rev_tl_op & loan_amnt